In [40]:
# Import des bibliothèques utilisées

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from lime import lime_tabular
import string
import nltk
from nltk.corpus import stopwords
import random
from sklearn.naive_bayes import MultinomialNB


In [41]:
# Data Preprocessing
# On rend analysable les textes : on passe les caractères en minuscule, on enlève la ponctuation etc.

def preprocessing(fake=pd.read_csv("True.csv"),true=pd.read_csv("True.csv")):
    fake['target'] = 'fake'
    true['target'] = 'true'
    data = pd.concat([fake, true]).reset_index(drop = True)
    data.drop(["date"],axis=1,inplace=True)
    data.drop(["title"],axis=1,inplace=True)
    data['text'] = data['text'].apply(lambda x: x.lower())

    def punctuation_removal(text):
        all_list = [char for char in text if char not in string.punctuation]
        clean_str = ''.join(all_list)
        return clean_str

    data['text'] = data['text'].apply(punctuation_removal)
    nltk.download('stopwords')
    stop = stopwords.words('english')
    data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    #print('data lenght = ',len(data))
    #print('lenfake = ',lenfake)
    #print('lentrue = ',lentrue)
   
    return(data)

data=preprocessing()
#print(d)

[nltk_data] Downloading package stopwords to /Users/evan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
    # Selection de la taille des échantillons

    nbfake=5
    nbtrue=5
    lenfake=5
    lentrue=5

    '''
    if nbfake>=(lenfake-1):
        print('Not enough fake news in dataset')
    if nbtrue>=(lentrue-1):
        print('Not enough true news in dataset')
    '''

    print(len(data))
    a=random.sample(range(0, lenfake), nbfake)
    b=random.sample(range(lenfake+1, len(data)-1), nbtrue)
    #print(b)
    print(' b = ',b)
    d=data.iloc[a][:]
    print(" a = ", a)

    print("data = ",data)
    e=data.iloc[b][:]
    
    data2=pd.concat([d,e], axis=0)

# Formation des echantillons d'entrainement et de test
    X_train,X_test,y_train,y_test = train_test_split(data2['text'],data2.target,test_size=0.2,random_state=42)

print('y_test = ',y_test)


#data2=dataproc(preprocessing(),nbfake,nbtrue)
#print(data2[0])


42834
 b =  [8760, 12145, 34798, 33861, 39108]
 a =  [0, 4, 2, 3, 1]
data =                                                      text       subject target
0      washington reuters head conservative republica...  politicsNews   fake
1      washington reuters transgender people allowed ...  politicsNews   fake
2      washington reuters special counsel investigati...  politicsNews   fake
3      washington reuters trump campaign adviser geor...  politicsNews   fake
4      seattlewashington reuters president donald tru...  politicsNews   fake
...                                                  ...           ...    ...
42829  brussels reuters nato allies tuesday welcomed ...     worldnews   true
42830  london reuters lexisnexis provider legal regul...     worldnews   true
42831  minsk reuters shadow disused sovietera factori...     worldnews   true
42832  moscow reuters vatican secretary state cardina...     worldnews   true
42833  jakarta reuters indonesia buy 11 sukhoi fighte...     worl

In [43]:
'''def subjects(data):
    print(data.groupby(['subject'])['text'].count())
    data.groupby(['subject'])['text'].count().plot(kind="bar")
    plt.show()

subjects(dataproc(data2,13,14))'''

'def subjects(data):\n    print(data.groupby([\'subject\'])[\'text\'].count())\n    data.groupby([\'subject\'])[\'text\'].count().plot(kind="bar")\n    plt.show()\n\nsubjects(dataproc(data2,13,14))'

In [44]:
# Fonction de vectorisation des textes

def vecto(X_train,X_test):
    
    X_t=pd.concat([X_train,X_test])
    #print('X_t = ',X_t.shape)
    ltotal=len(X_t)
    #print(X_t)
    vectorizer = TfidfVectorizer(lowercase=False)
    v = vectorizer.fit_transform(X_t)

    features=vectorizer.get_feature_names_out()
    ltrain=len(X_train)
    vtrain=v[0:ltrain][:]
    vtest=v[ltrain:][:]
    vtrain=scipy.sparse.csr_matrix.toarray(vtrain)
    vtest=scipy.sparse.csr_matrix.toarray(vtest)

    return(vtrain,vtest,vectorizer,features)
    

#print(vecto(data2[0],data2[1]))

In [45]:
# Fonction de test des performances de la classification par Regression Logistique

def classifLogReg(vtrain,vtest,y_train,y_test):
    mo=LogisticRegression()
    mo.fit(vtrain,y_train)
    m=mo.predict(vtest)
    m=np.array(m)
    y_test=np.array(y_test)
    #print(y_test)
    #print(m)

    #print("accuracy: {}%".format(round(accuracy_score(y_test, m)*100,2)))
    return(round(accuracy_score(y_test, m)*100,2))
print(classifLogReg)

<function classifLogReg at 0x7f91391c9430>


In [46]:
# Fonction classifiant les news en utilisant le modèle Bayesien naïf a laquelle on a branché l'outil d'explicabilité Lime. Retourne à partir des données les résultats de classification,
# sa performance et les mots constituant les news ayant eu la plus grande incidence sur le resultat de la classification.

def classifNaiveBayes(vtrain,y_train,vectorizer,X_test,features):
    mo=MultinomialNB()
    mo.fit(vtrain,y_train)

    i=X_test.index
    j=i[1]

    o=pd.Series(X_test[j])
    o=vectorizer.transform(o)
    o=scipy.sparse.csr_matrix.toarray(o)
    o=o[0]

    explainer = lime_tabular.LimeTabularExplainer(vtrain,mode="classification",categorical_features=features,feature_names=features)
    exp=explainer.explain_instance(data_row=o,predict_fn=mo.predict_proba, num_features=20) #labels=features
    exp.as_pyplot_figure()
    from matplotlib import pyplot as plt
    plt.tight_layout()
    exp.show_in_notebook()
    return(j,exp)

In [47]:
'''print(features)
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(8,6))
    plt.barh(range(len(mo.coef_[0])), mo.coef_[0], color=["red" if coef<0 else "green" for coef in mo.coef_[0]])
    plt.yticks(range(len(mo.coef_[0])), features);
    plt.title("Weights")
    '''

'print(features)\nwith plt.style.context("ggplot"):\n    fig = plt.figure(figsize=(8,6))\n    plt.barh(range(len(mo.coef_[0])), mo.coef_[0], color=["red" if coef<0 else "green" for coef in mo.coef_[0]])\n    plt.yticks(range(len(mo.coef_[0])), features);\n    plt.title("Weights")\n    '

In [48]:
'''def limenaivebayes(nbfake,nbtrue):
    data,lenfake,lentrue=preprocessing()
    
    x1=dataproc(data,nbfake,nbtrue,lenfake,lentrue)
    X_train=x1[0]
    X_test=x1[1]
    y_train=x1[2]
    y_test=x1[3]
    vtrain,vtest,vectorizer,features=vecto(X_train,X_test)
    #classifNaiveBayes(vtrain,y_train,vectorizer,X_test,features)
    j,exp=classifNaiveBayes(vtrain,y_train,vectorizer,X_test,features)
    #print('Texte classé = ',X_test[j])

    outputs = open("outputs.txt", "w")
    outputs.write("Texte classé = ")
    outputs.write(X_test[j])
    #outputs.write(exp)
    outputs.write("/n")
    outputs.close()
    
limenaivebayes(5,5)

'''


'def limenaivebayes(nbfake,nbtrue):\n    data,lenfake,lentrue=preprocessing()\n    \n    x1=dataproc(data,nbfake,nbtrue,lenfake,lentrue)\n    X_train=x1[0]\n    X_test=x1[1]\n    y_train=x1[2]\n    y_test=x1[3]\n    vtrain,vtest,vectorizer,features=vecto(X_train,X_test)\n    #classifNaiveBayes(vtrain,y_train,vectorizer,X_test,features)\n    j,exp=classifNaiveBayes(vtrain,y_train,vectorizer,X_test,features)\n    #print(\'Texte classé = \',X_test[j])\n\n    outputs = open("outputs.txt", "w")\n    outputs.write("Texte classé = ")\n    outputs.write(X_test[j])\n    #outputs.write(exp)\n    outputs.write("/n")\n    outputs.close()\n    \nlimenaivebayes(5,5)\n\n'

In [51]:
#data=preprocessing()
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Outil permettant d'extraire d'un texte le sentiment dominant (positif, négatif, neutre)
analyzer=SentimentIntensityAnalyzer()

print("X_train = ",X_train)
print('data = ',X_train[0:5])
NEG=[]
NEU=[]
POS=[]
COM=[]

for news in X_train:
    neg=analyzer.polarity_scores(news)['neg']
    neu=analyzer.polarity_scores(news)['neu'] 
    pos=analyzer.polarity_scores(news)['pos']
    com=analyzer.polarity_scores(news)['compound']
    NEG=NEG+[neg]
    NEU=NEU+[neu]
    POS=POS+[pos]
    COM=COM+[com]
    
A=NEG,NEU,POS,COM
sentitrain=np.array(A)
sentitrain=np.transpose(sentitrain)
sentitrain=sentitrain+1 #MultinomialNB ne prend que des matrices X>0 
print('sentitrain = ',sentitrain)


NEG=[]
NEU=[]
POS=[]
COM=[]
for news in X_test:
    neg=analyzer.polarity_scores(news)['neg']
    neu=analyzer.polarity_scores(news)['neu'] 
    pos=analyzer.polarity_scores(news)['pos']
    com=analyzer.polarity_scores(news)['compound']
    NEG=NEG+[neg]
    NEU=NEU+[neu]
    POS=POS+[pos]
    COM=COM+[com]

sentitest=NEG,NEU,POS,COM
sentitest=np.array(sentitest)
sentitest=np.transpose(sentitest)
sentitest=sentitest+1 #MultinomialNB ne prend que des matrices X>0
print('sentitest = ',sentitest)


'''
positive sentiment: compound score >= 0.05
neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
negative sentiment: compound score <= -0.05
'''

mo=MultinomialNB()
mo.fit(sentitrain,y_train)

i=X_test.index
j=i[1]

o=pd.Series(X_test[j])

vectorizer = TfidfVectorizer(lowercase=False)
vectorizer = vectorizer.fit(pd.concat([X_train,X_test]))

o=vectorizer.transform(o)
o=scipy.sparse.csr_matrix.toarray(o) # transformation de l'objet csr_matrix en numpy array, type d'entree de Lime
o=o[0]

features={neg,neu,pos,com}
print("o=",o)

explainer = lime_tabular.LimeTabularExplainer(sentitrain,mode="classification",categorical_features=features,feature_names=features)
# Utilisation de Lime sur le modèle entrainé

exp=explainer.explain_instance(data_row=o,predict_fn=mo.predict_proba, num_features=20) #labels=features
# Explication de la classification d'une news par les mots ayant eu le plus de poids dans sa classification par le modèle
exp.as_pyplot_figure()
from matplotlib import pyplot as plt
plt.tight_layout()
exp.show_in_notebook()
# Fonction permettant de montrer l'interprétation de Lime sur la classification d'une news


X_train =  8760     reuters scores photographers 2016 us president...
0        washington reuters head conservative republica...
34798    buenos aires reuters argentina given rescuing ...
2        washington reuters special counsel investigati...
39108    ankara reuters turkey president tayyip erdogan...
1        washington reuters transgender people allowed ...
3        washington reuters trump campaign adviser geor...
12145    united nationsyangon reuters us secretary stat...
Name: text, dtype: object
data =  8760     reuters scores photographers 2016 us president...
0        washington reuters head conservative republica...
34798    buenos aires reuters argentina given rescuing ...
2        washington reuters special counsel investigati...
39108    ankara reuters turkey president tayyip erdogan...
Name: text, dtype: object
sentitrain =  [[1.005  1.878  1.118  1.9805]
 [1.064  1.813  1.123  1.9801]
 [1.052  1.833  1.114  1.886 ]
 [1.072  1.844  1.084  1.3947]
 [1.244  1.652  1.104  0

KeyError: 4